In [1]:
import sys
import os

PROJECT_ROOT = os.path.dirname(os.path.abspath(os.getcwd()))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [2]:
from src.model import run_experiment

/home/master-y/Documents/FnCL/NLP/fast-clara4absa/torch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
run_experiment()

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /microsoft/Phi-3.5-mini-instruct/resolve/main/tokenizer_config.json (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))"), '(Request ID: a5fb0dde-6ba5-45e7-a60b-8930eaf1b2f7)')' thrown while requesting HEAD https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
`torch_dtype` is deprecated! Use `dtype` instead!
Epoch 0:   0%|                                                                                                                     | 0/9890 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/master-y/Documents/FnCL/NLP/fast-clara4absa/torch_env/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passe

KeyboardInterrupt: 